# Getting Started with StockSimulator

This notebook introduces the basics of backtesting trading strategies using StockSimulator.

## What You'll Learn
1. Loading historical data
2. Creating a simple strategy
3. Running a backtest
4. Analyzing results

## Setup

First, let's import the necessary modules.

In [ ]:
import sys
import os
from datetime import date

# Add parent directory to path
sys.path.insert(0, os.path.abspath('..'))

from stocksimulator.data import load_from_csv
from stocksimulator.core.backtester import Backtester
from stocksimulator.strategies import Balanced6040Strategy

## 1. Load Historical Data

Let's load S&P 500 historical data from our CSV file.

In [ ]:
# Load SPY data
spy_data = load_from_csv('sp500_stooq_daily.csv', 'SPY', '../historical_data')

print(f"Loaded {len(spy_data.data)} data points")
print(f"Date range: {spy_data.data[0].date} to {spy_data.data[-1].date}")
print(f"\nFirst data point:")
print(f"  Date: {spy_data.data[0].date}")
print(f"  Close: ${spy_data.data[0].close:.2f}")
print(f"\nLast data point:")
print(f"  Date: {spy_data.data[-1].date}")
print(f"  Close: ${spy_data.data[-1].close:.2f}")

## 2. Create a Simple Strategy

Let's create a simple buy-and-hold strategy that invests 100% in SPY.

In [ ]:
def buy_and_hold_strategy(current_date, market_data, portfolio, current_prices):
    """
    Simple buy-and-hold strategy.
    
    Returns target allocation: 100% SPY
    """
    return {'SPY': 100.0}

## 3. Run Backtest

Now let's run a backtest over the last 5 years.

In [ ]:
# Set backtest period (last 5 years)
end_date = spy_data.data[-1].date
start_date = date(end_date.year - 5, end_date.month, end_date.day)

print(f"Backtest period: {start_date} to {end_date}")

# Create backtester
backtester = Backtester(
    initial_cash=100000.0,  # Start with $100,000
    transaction_cost_bps=2.0  # 2 basis points per trade
)

# Run backtest
result = backtester.run_backtest(
    strategy_name='Buy & Hold SPY',
    market_data={'SPY': spy_data},
    strategy_func=buy_and_hold_strategy,
    start_date=start_date,
    end_date=end_date,
    rebalance_frequency='monthly'
)

print("\n✓ Backtest complete!")

## 4. Analyze Results

Let's examine the performance metrics.

In [ ]:
summary = result.get_performance_summary()

print("PERFORMANCE SUMMARY")
print("=" * 60)
print(f"Initial Value:       ${100000:>12,.2f}")
print(f"Final Value:         ${result.equity_curve[-1].value:>12,.2f}")
print(f"Total Return:        {summary['total_return']:>12.2f}%")
print(f"Annualized Return:   {summary['annualized_return']:>12.2f}%")
print(f"Sharpe Ratio:        {summary['sharpe_ratio']:>12.3f}")
print(f"Max Drawdown:        {summary['max_drawdown']:>12.2f}%")
print(f"Volatility:          {summary['volatility']:>12.2f}%")
print(f"Number of Trades:    {summary['num_trades']:>12}")

## 5. Visualize Equity Curve (Optional)

If you have matplotlib installed, you can plot the equity curve.

In [ ]:
try:
    import matplotlib.pyplot as plt
    
    # Extract dates and values
    dates = [point.date for point in result.equity_curve]
    values = [point.value for point in result.equity_curve]
    
    # Create plot
    plt.figure(figsize=(12, 6))
    plt.plot(dates, values, linewidth=2)
    plt.title('Portfolio Value Over Time', fontsize=16)
    plt.xlabel('Date')
    plt.ylabel('Portfolio Value ($)')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
except ImportError:
    print("Install matplotlib to visualize: pip install matplotlib")

## Next Steps

Now that you've run your first backtest, try:
1. Loading different data (TLT for bonds, GLD for gold)
2. Creating a 60/40 portfolio strategy
3. Comparing multiple strategies
4. Running parameter optimization

Check out the other notebooks:
- `02_strategy_comparison.ipynb`: Compare multiple strategies
- `03_optimization.ipynb`: Optimize strategy parameters